In [521]:
import pyshacl
from rdflib import OWL, RDF, RDFS, SH, Graph, Namespace
import os
import rdflib

In [534]:
g = Graph()
g.parse('G36_SP223-v1.0.ttl', format = 'ttl')
g = g.skolemize()

In [535]:
def build_query(fragment, node):
    query_prefix = """
        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
        WHERE {
"""
    query = query_prefix
    query += '\t'
    query += fragment
    query += '\n'
    query += '\t'
    query += f'FILTER(?node = <{node}>)'
    query += '\n'
    query += '}'
    return query

In [536]:
query_fragments = {
        'hasValue' : '?node sh:hasValue ?hasValue .',
        'class': '?node sh:class ?class . ',
        'path': '?node sh:path ?path .',
        'next_node': '?node ( (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property | sh:node/sh:property | sh:qualifiedValueShape ) ?next_node .'
    }

def recursive_query(results, node):
    #Running different queries with each query fragment
    for key, fragment in query_fragments.items():
        query = build_query(fragment, node)
        res = g.query(query)
        # If results return for the query, it will be added to the results
        if (len(res.bindings)) > 0:
            # Most cases will have just 1 binding. If there are multiple bindings, we should iterate them
            for binding in res.bindings:
                # If the node has no results stored, we should add a dictionary entry for it 
                if results.get(binding.get('node')) is None:
                    results[binding.get('node')] = {}
                    results[binding.get('node')].update(binding)
                # if the specific value returned by the query is not in the dictionary entry, we should add it
                elif results.get(binding.get('node')).get(rdflib.term.Variable(key)) is None:
                    results[binding.get('node')].update(binding)
                # if the values returned by the query are already in the dictionary entry, they don't need to be added
                # display a message in this case 
                elif results.get(binding.get('node')).get(rdflib.term.Variable(key)) == binding.get(rdflib.term.Variable(key)):
                    print('values are the same')
                    print(len(results.items()))
                    if len(results.items()) > 6:
                        return results
                else:
                # with and/or/multiple property queries, there may be multiple values, 
                # each next node should be dictionary of its own, chaining
                    print('values are different')
                if binding.get('next_node', None):
                    # update to change structure
                    next_node = binding.get('next_node')
                    recursive_query(results[binding.get('node')], next_node)
                    

                    # node = binding.get('next_node')
                    # results[node] = {}
                    # recursive_query(results[node], node)
        
    return results

def convert_to_str(val_dict):
    # more recursion haha
    new_dict = {}
    for k, v in val_dict.items():
        if isinstance(v, dict):
            new_dict[str(k)] = convert_to_str(v)
        else:
            new_dict[str(k)] = str(v)
    return new_dict

In [537]:
results = {}
result_list = [results]
node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'

value_returned = recursive_query(results, node)
results = convert_to_str(value_returned)
display(results)

{'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil': {'node': 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil',
  'class': 'http://data.ashrae.org/standard223#ResistanceHeater',
  'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb41',
  'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb41': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb41',
   'path': 'http://data.ashrae.org/standard223#hasProperty',
   'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb42',
   'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb42': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb42',
    'class': 'http://data.ashrae.org/standard223#QuantifiableActuatableProperty',
    'next_node': 'https://rdflib.

In [556]:

def get_node_name_map(val_dict, node_name_map = {}, i= 0):
    for k, v in val_dict.items():
        i += 1
        if isinstance(v, dict):
            get_node_name_map(v, node_name_map, i)
            node_name_map[k] = f"node_{i}"
        elif k in node_name_map:
            continue
        elif 'node' in k:
            node_name_map[v] = f"node_{i}"
        else:
            continue
    return node_name_map
node_name_map = {}
i= 0 = get_node_name_map(results, node_name_map, i)
        

http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil
node
class
here
next_node
https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb41
node
path
here
next_node
https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb42
node
class
here
next_node
https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb44
node
hasValue
here
path
here


In [557]:
a

{'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil': 'node_1',
 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb41': 'node_5',
 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb42': 'node_9',
 'https://rdflib.github.io/.well-known/genid/rdflib/n6218e3a7dba9477e810c8628e45d47bcb44': 'node_13'}

In [526]:
def sequential_numbers(start=0, step=1):
    number = start
    while True:
        yield number
        number += step

In [527]:
def sequential_numbers(start=0, step=1):
    number = start
    while True:
        yield number
        number += step

def dict_traverser(val_dict):
    for k, v in val_dict.items():
        if isinstance(v, dict):
            yield from dict_traverser(v)
        else:
            continue
    yield v

node_seq = sequential_numbers()

def create_node_name_map(results):
    dict_generator = dict_traverser(results)
    node_name_map = {}
    for val_dict in dict_generator:
        if not isinstance(val_dict, dict):
            continue
        for k, v in val_dict.items():
            if k in node_name_map:
                continue
            elif 'node' in k:
                node_name_map[v] = f"node_{next(node_seq)}"
            elif isinstance(v, dict):
                node_name_map[k] = f"node_{next(node_seq)}"
    return node_name_map

In [528]:
# global previous_node
previous_node = 0
# global temp_query 
temp_query = """
        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        """

def create_query_line(node_dict):
    global temp_query
    global previous_node
    if 'class' in node_dict.keys():
        print('here')
        temp_query += '\t'
        temp_query += f"?{node_dict.get('node')} a <{node_dict.get('class')}> ."
        temp_query += '\n'
    if 'path' in node_dict.keys():
        temp_query += '\t'
        temp_query += f"?{previous_node} <{node_dict.get('path')}> "
        if node_dict.get('hasValue'):
            temp_query+= f"<{node_dict.get('hasValue')}> ."
        else:
            temp_query+= f"?{node_dict.get('next_node')}."
        temp_query += '\n'
    previous_node = node_dict.get('node')
    return 
    

def construct(node_dict):
    # more recursion haha
    create_query_line(node_dict)
    for k, v in node_dict.items():
        if isinstance(v, dict):
            construct(v)
        else:
            continue
    return 

In [529]:
node_name_map = create_node_name_map(results)

In [530]:
def convert_with_map(val_dict):
    new_dict = {}
    for k, v in val_dict.items():
        if isinstance(v, dict):
            new_dict[node_name_map[k]] = convert_with_map(v)
        elif v in node_name_map.keys(): 
            new_dict[k] = node_name_map[v]
        else:
            new_dict[k] = v
    return new_dict

In [531]:
construct(convert_with_map(results))
temp_query += '}'
print(temp_query)

here
here

        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        	?node_5 a <http://data.ashrae.org/standard223#ResistanceHeater> .
	?node_5 <http://data.ashrae.org/standard223#hasProperty> ?node_4.
	?node_4 a <http://data.ashrae.org/standard223#QuantifiableActuatableProperty> .
	?node_4 <http://qudt.org/schema/qudt/hasQuantityKind> <http://qudt.org/vocab/quantitykind/DimensionlessRatio> .
}


In [300]:
test = Graph()
test.parse('test.ttl', format = 'ttl')
res = test.query(temp_query)

In [301]:
for r in res:
    print(r)

## With VAV

In [313]:
results = {}
result_list = [results]
node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#VAV'

value_returned = recursive_query(results, node)
results = convert_to_str(value_returned)
node_name_map = create_node_name_map(results)
results = convert_with_map(results)
display(results)

values are different


{'node_22': {'node': 'node_22',
  'class': 'http://data.ashrae.org/standard223#SingleDuctTerminal',
  'next_node': 'node_23',
  'node_23': {'node': 'node_23',
   'path': 'http://data.ashrae.org/standard223#contains',
   'next_node': 'node_12',
   'node_12': {'node': 'node_12',
    'class': 'http://data.ashrae.org/standard223/1.0/extensions/g36#Damper'}},
  'node_20': {'node': 'node_20',
   'path': 'http://data.ashrae.org/standard223#hasConnectionPoint',
   'next_node': 'node_21',
   'node_21': {'node': 'node_21',
    'class': 'http://data.ashrae.org/standard223#OutletConnectionPoint',
    'next_node': 'node_19',
    'node_19': {'node': 'node_19',
     'path': 'http://data.ashrae.org/standard223#hasProperty',
     'next_node': 'node_17',
     'node_17': {'node': 'node_17',
      'class': 'http://data.ashrae.org/standard223#QuantifiableObservableProperty',
      'next_node': 'node_15',
      'node_15': {'node': 'node_15',
       'hasValue': 'http://qudt.org/vocab/quantitykind/VolumeFlowR

In [314]:
def sequential_numbers(start=0, step=1):
    number = start
    while True:
        yield number
        number += step

def dict_traverser(val_dict):
    for k, v in val_dict.items():
        if isinstance(v, dict):
            yield from dict_traverser(v)
        else:
            continue
    yield v

node_seq = sequential_numbers()

def create_node_name_map(results):
    dict_generator = dict_traverser(results)
    node_name_map = {}
    for val_dict in dict_generator:
        if not isinstance(val_dict, dict):
            continue
        for k, v in val_dict.items():
            if k in node_name_map:
                continue
            elif 'node' in k:
                node_name_map[v] = f"node_{next(node_seq)}"
            elif isinstance(v, dict):
                node_name_map[k] = f"node_{next(node_seq)}"
    return node_name_map
node_name_map = create_node_name_map(results)

In [315]:

# global previous_node
previous_node = 0
# global temp_query 
temp_query = """
        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        """

def create_query_line(node_dict):
    global temp_query
    global previous_node
    if 'class' in node_dict.keys():
        temp_query += '\t'
        temp_query += f"?{node_dict.get('node')} a <{node_dict.get('class')}> ."
        temp_query += '\n'
    if 'path' in node_dict.keys():
        temp_query += '\t'
        temp_query += f"?{previous_node} <{node_dict.get('path')}> "
        if node_dict.get('hasValue'):
            temp_query+= f"<{node_dict.get('hasValue')}> ."
        else:
            temp_query+= f"?{node_dict.get('next_node')}."
        temp_query += '\n'
    previous_node = node_dict.get('node')
    return 
    

def construct(node_dict):
    # more recursion haha
    create_query_line(node_dict)
    for k, v in node_dict.items():
        if isinstance(v, dict):
            construct(v)
        else:
            continue
    return 

construct(results)
temp_query += '}'

In [316]:
print(temp_query)


        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        	?node_22 a <http://data.ashrae.org/standard223#SingleDuctTerminal> .
	?node_22 <http://data.ashrae.org/standard223#contains> ?node_12.
	?node_12 a <http://data.ashrae.org/standard223/1.0/extensions/g36#Damper> .
	?node_12 <http://data.ashrae.org/standard223#hasConnectionPoint> ?node_21.
	?node_21 a <http://data.ashrae.org/standard223#OutletConnectionPoint> .
	?node_21 <http://data.ashrae.org/standard223#hasPropert

In [317]:
results

{'node_22': {'node': 'node_22',
  'class': 'http://data.ashrae.org/standard223#SingleDuctTerminal',
  'next_node': 'node_23',
  'node_23': {'node': 'node_23',
   'path': 'http://data.ashrae.org/standard223#contains',
   'next_node': 'node_12',
   'node_12': {'node': 'node_12',
    'class': 'http://data.ashrae.org/standard223/1.0/extensions/g36#Damper'}},
  'node_20': {'node': 'node_20',
   'path': 'http://data.ashrae.org/standard223#hasConnectionPoint',
   'next_node': 'node_21',
   'node_21': {'node': 'node_21',
    'class': 'http://data.ashrae.org/standard223#OutletConnectionPoint',
    'next_node': 'node_19',
    'node_19': {'node': 'node_19',
     'path': 'http://data.ashrae.org/standard223#hasProperty',
     'next_node': 'node_17',
     'node_17': {'node': 'node_17',
      'class': 'http://data.ashrae.org/standard223#QuantifiableObservableProperty',
      'next_node': 'node_15',
      'node_15': {'node': 'node_15',
       'hasValue': 'http://qudt.org/vocab/quantitykind/VolumeFlowR

In [318]:
# At a junction, the ordering of the results will mess up the query form
# the node before the first junction should be the node before the second junction
dict_junctions = []
def get_dict_junctions(node_dict):
    # more recursion haha
    j = 0
    for k, v in node_dict.items():
        if isinstance(v, dict):
            if j > 0:
                dict_junctions.append(dict_junction)
                dict_junctions.append(k)
            j = 1
            get_dict_junctions(v)
            dict_junction = k
        else:
            continue
    return 
get_dict_junctions(results)


In [319]:
dict_junctions

['node_23', 'node_20']

In [393]:
input = {
    'a': 1,
    'b': {
        'b1': 'b1_val',
        'b2': {
            'b21': {
                'b211': 'b211_val',
                'b212': 'b212_val'
            },
            'b22': {
                'b221': 'b221_val',
                'b222': 'b222_val'
            },
            'b23': {
                'b231': 'b221_val',
                'b232': 'b222_val'
            }
        },
        'b3': {
            'b31': {
                'b311': 'b311_val',
                'b312': 'b312_val'
            },
            'b32': {
                'b321': 'b321_val',
                'b322': 'b322_val'
            }
        },
    }
    }

In [417]:
def get_parents(node_dict):
    j = 0
    for k, v in node_dict.items():
        if isinstance(v, dict):
            if set(v.keys()).intersection(dict_junctions):
                parents.append(k)
            get_parents(v)
        else:
            continue
    return 


In [418]:
dict_junctions= []
parents = []
get_dict_junctions(input)
dict_junctions

['b21', 'b22', 'b22', 'b23', 'b2', 'b3', 'b31', 'b32']

In [419]:
get_parents(input)

In [560]:
parents

['b2', 'b3', 'b']

In [561]:
dict_junctions = []
def get_dict_junctions_with_parents(node_dict):
    # more recursion haha
    j = 0
    for k, v in node_dict.items():
        if isinstance(v, dict):
            if j == 1:
                if dict_junction:
                    dict_junctions.append(dict_junction)
                    dict_junction = None
                dict_junctions.append(k)
            if j == 0:
                dict_junction = k
            j = 1
            get_dict_junctions_with_parents(v)
            if set(v.keys()).intersection(dict_junctions):
                parent_dict[k] = set(v.keys()).intersection(dict_junctions)
        else:
            continue
    return 

In [562]:
dict_junctions= []
parent_dict = {}
get_dict_junctions_with_parents(input)
display(dict_junctions)
display(parents)

['b21', 'b22', 'b23', 'b2', 'b3', 'b31', 'b32']

['b2', 'b3', 'b']

In [458]:
# I can use the parent nodes to capture the previous node that should be used in the query generation
# b2 would be the previous node used for b21, b22, and b23
parent_dict

{'b2': {'b21', 'b22', 'b23'}, 'b3': {'b31', 'b32'}, 'b': {'b2', 'b3'}}

## With Zone, reinterpretting into single property queries

In [564]:
results = {}
result_list = [results]
node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#Zone'

value_returned = recursive_query(results, node)
results = convert_to_str(value_returned)
node_name_map = create_node_name_map(results)
results = convert_with_map(results)
display(results)

values are different
values are different
values are different
values are different
values are different
values are different
values are different
values are different
values are different


{'node_27': {'node': 'node_27',
  'class': 'http://data.ashrae.org/standard223#Zone',
  'next_node': 'node_28',
  'node_28': {'node': 'node_28',
   'path': 'http://data.ashrae.org/standard223#hasProperty',
   'next_node': 'node_8',
   'node_8': {'node': 'node_8',
    'class': 'http://data.ashrae.org/standard223#QuantifiableObservableProperty',
    'next_node': 'node_9',
    'node_9': {'node': 'node_9',
     'hasValue': 'http://qudt.org/vocab/quantitykind/Temperature',
     'path': 'http://qudt.org/schema/qudt/hasQuantityKind'},
    'node_7': {'node': 'node_7',
     'hasValue': 'http://data.ashrae.org/standard223#Dimensioned-Delta',
     'path': 'http://data.ashrae.org/standard223#hasAspect'}}},
  'node_29': {'node': 'node_29',
   'path': 'http://data.ashrae.org/standard223#hasProperty',
   'next_node': 'node_11',
   'node_11': {'node': 'node_11',
    'class': 'http://data.ashrae.org/standard223#EnumeratedObservableProperty',
    'next_node': 'node_12',
    'node_12': {'node': 'node_12'

In [565]:
dict_junctions= []
parent_dict = {}
get_dict_junctions_with_parents(results)
display(dict_junctions)
display(parent_dict)


['node_9',
 'node_7',
 'node_28',
 'node_29',
 'node_30',
 'node_15',
 'node_13',
 'node_31',
 'node_18',
 'node_16',
 'node_32',
 'node_33',
 'node_25']

{'node_8': {'node_7', 'node_9'},
 'node_14': {'node_13', 'node_15'},
 'node_17': {'node_16', 'node_18'},
 'node_27': {'node_25',
  'node_28',
  'node_29',
  'node_30',
  'node_31',
  'node_32',
  'node_33'}}

In [571]:
def get_node_parent(parent_dict, node_name):
    for k, v in parent_dict.items():
        if isinstance(v, dict):
            get_node_parent(v)
        elif isinstance(v, set):
            if node_name in v:
                parent_node_name = k
        else:
            continue
    return parent_node_name

In [572]:
get_node_parent(parent_dict, 'node_25')

'node_27'

In [573]:
def get_node_parent(node_name, parent_dict):
    _, _, parent_node_name = _get_node_parent(parent_dict, node_name)
    return parent_node_name
def _get_node_parent(parent_dict, node_name, parent_node_name = ''):
    for k, v in parent_dict.items():
        if isinstance(v, dict):
            _get_node_parent(v, parent_node_name)
        elif isinstance(v, set):
            if node_name in v:
                parent_node_name = k
        else:
            continue
    return parent_dict, node_name, parent_node_name

In [574]:
def get_node_parent(node_name, parent_dict):
    _, _, parent_node_name = _get_node_parent(parent_dict, node_name)
    return parent_node_name


In [575]:
node_name = 'node_56'
parent_node_name = 'this'
parent_node_name = get_node_parent(node_name, parent_dict)

In [576]:
parent_node_name

''

In [577]:
# node_seq = sequential_numbers()
# node_name_map = create_node_name_map(results)

In [578]:
# global previous_node
previous_node = 0
# global temp_query 
temp_query = """
        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        """

def create_query_line(node_dict):
    global temp_query
    global previous_node
    if 'class' in node_dict.keys():
        temp_query += '\t'
        temp_query += f"?{node_dict.get('node')} a <{node_dict.get('class')}> ."
        temp_query += '\n'
    if 'path' in node_dict.keys():
        if node_dict.get('node') in dict_junctions:
            previous_node = get_node_parent(node_dict.get('node'), parent_dict)
        temp_query += '\t'
        temp_query += f"?{previous_node} <{node_dict.get('path')}> "
        if node_dict.get('hasValue'):
            temp_query+= f"<{node_dict.get('hasValue')}> ."
        else:
            temp_query+= f"?{node_dict.get('next_node')}."
        temp_query += '\n'
    previous_node = node_dict.get('node')
    return 
    

def construct(node_dict):
    # more recursion haha
    create_query_line(node_dict)
    for k, v in node_dict.items():
        if isinstance(v, dict):
            construct(v)
        else:
            continue
    return 

construct(results)
temp_query += '}'

In [579]:
print(temp_query)


        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        	?node_27 a <http://data.ashrae.org/standard223#Zone> .
	?node_27 <http://data.ashrae.org/standard223#hasProperty> ?node_8.
	?node_8 a <http://data.ashrae.org/standard223#QuantifiableObservableProperty> .
	?node_8 <http://qudt.org/schema/qudt/hasQuantityKind> <http://qudt.org/vocab/quantitykind/Temperature> .
	?node_8 <http://data.ashrae.org/standard223#hasAspect> <http://data.ashrae.org/standard223#Dimensioned-Del

In [514]:
def delete_unhandled_paths(query):
    query_list = temp_query.split('\n')
    new_query = ''
    for line in query_list:
        if '?None' in line:
            print(line)
            print('Unhandled line, likely due to complex property path')
        else:
            new_query += '\n'
            new_query += line
    return new_query
    

In [515]:
new_query = delete_unhandled_paths(temp_query)

	?node_40 <https://rdflib.github.io/.well-known/genid/rdflib/n0a5634c387ec4df0835e2eb53f0b38fdb93> ?None.
Unhandled line, likely due to complex property path


In [517]:
print(new_query)



        PREFIX s223: <http://data.ashrae.org/standard223#>
        PREFIX unit: <http://qudt.org/vocab/unit/>
        PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
        PREFIX qudt: <http://qudt.org/schema/qudt/>
        PREFIX sh: <http://www.w3.org/ns/shacl#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

        SELECT *
            WHERE {
        	?node_59 a <http://data.ashrae.org/standard223#Zone> .
	?node_59 <http://data.ashrae.org/standard223#hasProperty> ?node_40.
	?node_40 a <http://data.ashrae.org/standard223#EnumeratedObservableProperty> .
	?node_41 a <http://data.ashrae.org/standard223#Window> .
	?node_40 <http://data.ashrae.org/standard223#hasEnumerationKind> <http://data.ashrae.org/standard223#EnumerationKind-OnOff> .
	?node_59 <http://data.ashr

# Experimenting

In [648]:
temp_query = ""
for k, v in results.items():
    for k, v in results.items():
        print(v)
        if 'class' in v.keys():
            print('here')
            temp_query += '\t'
            temp_query += f"?{v.get('node')} a <{v.get('class')}> ."
            temp_query += '\n'
        previous_node = v.get('node')

        if 'path' in v.keys():
            temp_query += '\t'
            temp_query += f"?{previous_node} {v.get('path')} "
            if 'hasValue' in v.keys():
                temp_query+= f"{results.get('hasValue')} ."
            elif results.get('next_node'):
                temp_query+= f"? {results.get('next_node')}."
            else:
                # just constraining that it has this path
                print('no value or next node')
                temp_query+= f"next_node_for_path."
            temp_query += '\n'
        


{'node': 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil', 'class': 'http://data.ashrae.org/standard223#ResistanceHeater', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41', 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41', 'path': 'http://data.ashrae.org/standard223#hasProperty', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42', 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42', 'class': 'http://data.ashrae.org/standard223#QuantifiableActuatableProperty', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44', 'https://rdflib.github.io/.

In [12]:
""" results could be converted to query as 
node1 a resistance heater
node1 hasProperty node3
node3 a quantifiableacutatableproperty 
node3 hasQuantityKind hasValue
"""


' results could be converted to query as \nnode1 a resistance heater\nnode1 hasProperty node3\nnode3 a quantifiableacutatableproperty \nnode3 hasQuantityKind hasValue\n'

In [13]:
node_dict = results.copy()
for i in range(0,10):
    for k, v in node_dict.items():
        if isinstance(v, dict):
            node_dict = v 
            print(v)
            break

{'node': 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil', 'class': 'http://data.ashrae.org/standard223#ResistanceHeater', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b41', 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b41': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b41', 'path': 'http://data.ashrae.org/standard223#hasProperty', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b42', 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b42': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b42', 'class': 'http://data.ashrae.org/standard223#QuantifiableActuatableProperty', 'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/n483a9c539c724f7b80937df3223463e8b44', 'https://rdflib.github.io/.

In [650]:
print(temp_query)

	?http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil a <http://data.ashrae.org/standard223#ResistanceHeater> .



In [ ]:
""" results could be converted to query as 
node1 a resistance heater
node1 hasProperty node3
node3 a quantifiableacutatableproperty 
node3 hasQuantityKind hasValue
"""


In [885]:
def convert_to_list_dicts(val_dict):
    dict_list = []
    for k, v in val_dict.items():
        amt = get_dict_amounts(v)
        if isinstance(v, dict):
            amt -= 1
            if amt == 0:
                new_dict[k] = convert_to_str(v)
        else:
            new_dict[str(k)] = str(v)
    return new_dict

def get_dict_amounts(d):
    i = 0
    for k, v in d.items():
        if isinstance(v, dict):
            i += 1
    return i
    

In [678]:
def construct(val_dict):
    # more recursion haha
    new_dict = {}
    for k, v in val_dict.items():
        if isinstance(v, dict):
            new_dict[str(k)] = construct(v)
            print(get_dict_amounts(v))
            print(k)
        else:
            new_dict[str(k)] = str(v)
    return new_dict

In [679]:
construct(results)

0
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44
1
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42
1
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41
1
http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil


{'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil': {'node': 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil',
  'class': 'http://data.ashrae.org/standard223#ResistanceHeater',
  'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41',
  'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41',
   'path': 'http://data.ashrae.org/standard223#hasProperty',
   'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42',
   'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42': {'node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42',
    'class': 'http://data.ashrae.org/standard223#QuantifiableActuatableProperty',
    'next_node': 'https://rdflib.

In [604]:
single_paths = convert_to_list_dicts([results])

KeyError: 3

In [605]:
display(single_paths)

[{'node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44',
  'class': 'http://data.ashrae.org/standard223#QuantifiableActuatableProperty',
  'next_node': 'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44',
  'path': 'http://qudt.org/schema/qudt/hasQuantityKind',
  'hasValue': 'http://qudt.org/vocab/quantitykind/DimensionlessRatio',
  'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44': {...},
  'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb43': {...},
  'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42': {...},
  'https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41': {...},
  'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil': {...}}]

In [510]:
for k, v in results['https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41'].items():
    print(k)

https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42


In [370]:
results = {}
result_list = [results]
node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
for i in range(0,6):
    #Running different queries with each query fragment
    for key, fragment in query_fragments.items():
        query = build_query(fragment, node)
        res = g.query(query)
        # If results return for the query, it will be added to the results
        if (len(res.bindings)) > 0:
            # Most cases will have just 1 binding. If there are multiple bindings, we should iterate them
            for binding in res.bindings:
                # If the node has no results stored, we should add a dictionary entry for it 
                if results.get(binding.get('node')) is None:
                    results[binding.get('node')] = {}
                    results[binding.get('node')].update(binding)
                # if the specific value returned by the query is not in the dictionary entry, we should add it
                elif results.get(binding.get('node')).get(rdflib.term.Variable(key)) is None:
                    results[binding.get('node')].update(binding)
                # if the values returned by the query are already in the dictionary entry, they don't need to be added
                # display a message in this case 
                elif results.get(binding.get('node')).get(rdflib.term.Variable(key)) == binding.get(rdflib.term.Variable(key)):
                    print('values are the same')
                else:
                # with and/or/multiple property queries, there may be multiple values, 
                # each next node should be dictionary of its own, chaining
                    print('values are different')
                if binding.get('next_node', None):
                    node = binding.get('next_node')
                    print(node)


https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb43
https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb44
values are the same
values are the same


In [371]:
results == value_returned

True

In [331]:
results
# results.get('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil')

{rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'): {rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'),
  rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater'),
  rdflib.term.Variable('next_node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41')},
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41'): {rdflib.term.Variable('node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41'),
  rdflib.term.Variable('path'): rdflib.term.URIRef('http://data.ashrae.org/standard223#hasProperty'),
  rdflib.term.Variable('next_node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42')},
 rdflib.term.URI

In [327]:
results.get(rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'))

rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater')

In [273]:
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
query = query_prefix
query += '\t'
query += query_fragments.get('next_node')
query += '\n'
query += '\t'
query += f'FILTER(?node = <{next_node}>)'
query += '\n'
query += '}'

In [274]:
query_typed = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

SELECT *
WHERE {
  ?node ( (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property | sh:node | sh:qualifiedValueShape ) ?object .
  FILTER(?node = <https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41>) 
}
"""

In [276]:
res =  g.query(query_typed)
res.bindings

[{rdflib.term.Variable('node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41'), rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb42')}]

In [258]:
results = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
for i in range(0,10):
    try:
        res = g.query(query)
        # next_node = res._bindings[0]['next_node']
        print(res.bindings)
        results.append(res)
    except Exception as e:
        print(i, e)


[{rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.Variable('next_node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41')}]
[{rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.Variable('next_node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41')}]
[{rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.Variable('next_node'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41')}]
[{rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.Variable('next_node'): rdflib.term

In [234]:
results

In [178]:
query_typed = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

SELECT *
WHERE {
  ?node ( (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property | sh:node | sh:qualifiedValueShape ) ?object .
  FILTER(?node = <http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil>) 
}
"""

In [239]:
res =  g.query(query_typed)
for r in res:
    print(r)

(rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41'))


In [240]:
print(res._bindings)

[{rdflib.term.Variable('node'): rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/nde5f9027b5594f6cb3ce243df260dfecb41')}]


In [237]:
res =  g.query(query)
print(res._bindings)

[]


In [236]:
res.vars

[rdflib.term.Variable('node'),
 rdflib.term.Variable('next_node'),
 rdflib.term.Variable('path'),
 rdflib.term.Variable('class'),
 rdflib.term.Variable('hasValue')]

In [160]:
i = 0
for res in results:
    for r in res._bindings:
        if r.get('class'):
            query += '\t'
            query += f"?n{i} a {r.get('class')} ."
            query += '\n'
        if r.get('path'):
            query += '\t'
            query += f"?n{i} {r.get('path')} "
            if r.get('value'):
                query+= f"{r.get('value')} ."
            else:
                query+= f"?n{i+1} ."
            query += '\n'
        i += 1
query += "}"

# Working on SPARQL Queries

In [117]:
# getting necessary pieces of query
query_prop_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?class ?path ?hasValue
    WHERE {
        OPTIONAL {?node (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .}
        OPTIONAL {?node sh:class ?class .} 
        OPTIONAL {?node sh:hasValue ?hasValue .} 

    }
    """
query_node_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?path ?class ?hasValue
    WHERE {
        OPTIONAL {?node (sh:or|sh:and)/rdf:rest*/rdf:first | sh:node | sh:qualifiedValueShape ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .} 
        OPTIONAL {?node sh:class ?class .}
        OPTIONAL {?node sh:hasValue ?hasValue .}  

    }
    """

In [118]:
# not sure why below aren't working
# optional seems to work strangely
# getting necessary pieces of query
query_node_inexplicably_working = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?path ?class ?hasValue
    WHERE {
        FILTER (?node = <%s>) 
        OPTIONAL {?node sh:path ?path } 
        OPTIONAL {?node sh:class ?class }
        OPTIONAL {?node sh:hasValue ?hasValue } 
        OPTIONAL {?node (sh:or|sh:and)/rdf:rest*/rdf:first | sh:node | sh:qualifiedValueShape ?object } 
    }
    """

In [119]:
subsequent_query = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT * WHERE {
        ?node rdfs:subClassOf ?what .
        OPTIONAL {?node sh:hasValue ?hasValue .}
        OPTIONAL {?node sh:class ?class .}
        OPTIONAL {?node sh:path ?path .}
        ?node sh:node*/sh:property ?next_node .
        FILTER(?node = <%s>) .
    }
"""

In [120]:
g = Graph()
g.parse('G36_SP223-v1.0.ttl', format = 'ttl')
g = g.skolemize()

In [121]:
results = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
for i in range(0,10):
    try:
        res = g.query(query_prop_all % next_node)
        res.serialize('res.csv',format = 'csv')
        next_node = res._bindings[0]['object']
        print(res._bindings)
        results.append(res)
    except Exception as e:
        print(i, e)
    try:
        res = g.query(query_node_all % next_node)
        res.serialize('res.csv',format = 'csv')
        print(res._bindings)
        next_node = res._bindings[0]['object']
        results.append(res)
    except Exception as e:
        print(i, e)
        


[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b41'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater')}]
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b42'), rdflib.term.Variable('path'): rdflib.term.URIRef('http://data.ashrae.org/standard223#hasProperty')}]
1 list index out of range
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b43'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#QuantifiableActuatableProperty')}]
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b44')}]
[]
2 list index out of range
3 list index out of range
[]
3 list ind

In [93]:
results = []
node_1 = []
node_2 = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
for i in range(0,10):
    try:
        res = g.query(query_prop_all % next_node)
        res.serialize('res.csv',format = 'csv')
        next_node_1 = res._bindings[0]['object']
        next_node = next_node_1
        print(res._bindings)
        results.append(res)
    except Exception as e:
        print(i, e)
    try:
        res = g.query(query_node_all % next_node)
        res.serialize('res.csv',format = 'csv')
        print(res._bindings)
        next_node_2 = res._bindings[0]['object']
        next_node = next_node_2
        results.append(res)
    except Exception as e:
        print(i, e)
    node_1.append(next_node_1)
    node_2.append(next_node_2)
    

[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b41'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater')}]
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b42'), rdflib.term.Variable('path'): rdflib.term.URIRef('http://data.ashrae.org/standard223#hasProperty')}]
1 list index out of range
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b43'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#QuantifiableActuatableProperty')}]
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44')}]
[]
2 list index out of range
3 list index out of range
[]
3 list ind

In [100]:
node_1

[rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b41'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b41'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44'),
 

In [130]:
results = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
for i in range(0,10):
    
    res = g.query(query_prop_all % next_node)
    if len(res._bindings) > 0:
        results.append(res)
        print(res._bindings)
        try:
            next_node = res._bindings[0]['object']
        except Exception as e:
            print(i, e)

    else:
        res = g.query(query_node_all % next_node)
        if len(res._bindings) > 0:
            results.append(res)
            print(res._bindings)
            try:
                next_node = res._bindings[0]['object']
            except Exception as e:
                print(i, e)
        else:
            res = g.query(query_node_inexplicably_working % next_node)
            if len(res._bindings) > 0:
                results.append(res)
                print(res._bindings)
                try:
                    next_node = res._bindings[0]['object']
                except Exception as e:
                    print(i, e)


In [131]:
g.serialize('res.ttl', format = 'ttl')

<Graph identifier=N2e71a1984ccc403c94038db79348b5b9 (<class 'rdflib.graph.Graph'>)>

In [132]:
for i, res in enumerate(results):
    for j, r in enumerate(res._bindings):
        print(i,j,r)

In [75]:
query = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

SELECT ?object ?path ?class
    WHERE {
"""
i = 0
for res in results:
    for r in res._bindings:
        if r.get('class'):
            query += '\t'
            query += f"?n{i} a {r.get('class')} ."
            query += '\n'
        if r.get('path'):
            query += '\t'
            query += f"?n{i} {r.get('path')} "
            if r.get('value'):
                query+= f"{r.get('value')} ."
            else:
                query+= f"?n{i+1} ."
            query += '\n'
        i += 1
query += "}"

In [77]:
print(query)


PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

SELECT ?object ?path ?class
    WHERE {
}


In [55]:
# THIS QUERY IS working as expected on the final nested node???
query_node_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT *
    WHERE {
        FILTER (?node = <%s>) 
        OPTIONAL {?node sh:path ?path } 
        OPTIONAL {?node sh:class ?class }
        OPTIONAL {?node sh:hasValue ?hasValue } 
        OPTIONAL {?node (sh:or|sh:and)/rdf:rest*/rdf:first | sh:node | sh:qualifiedValueShape ?object } 
    }
    """
res = g.query(query_node_all % 'https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b44')
res.serialize('res.csv',format = 'csv')

In [68]:
query_prop_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?class ?path ?hasValue
    WHERE {
        OPTIONAL {?node (sh:or|sh:and)/rdf:rest*/rdf:first | sh:property ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .}
        OPTIONAL {?node sh:class ?class .} 
        OPTIONAL {?node sh:hasValue ?hasValue .} 

    }
    """
res = g.query(query_prop_all % 'https://rdflib.github.io/.well-known/genid/rdflib/n1542809021084e79836531bd5bde9897b43')
res.serialize('res.csv',format = 'csv')

In [138]:
# MORE TARGETTED
# getting necessary pieces of query
query_prop_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?class ?path ?hasValue
    WHERE {
        OPTIONAL {?node sh:property ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .}
        OPTIONAL {?node sh:class ?class .} 
        OPTIONAL {?node sh:hasValue ?hasValue .} 

    }
    """
query_node_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?path ?class ?hasValue
    WHERE {
        OPTIONAL {?node sh:node ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .} 
        OPTIONAL {?node sh:class ?class .}
        OPTIONAL {?node sh:hasValue ?hasValue .}  

    }
    """
query_qualified_all = """
    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT ?object ?path ?class ?hasValue
    WHERE {
        OPTIONAL {?node sh:qualifiedValueShape ?object.}
        FILTER (?node = <%s>) .
        OPTIONAL {?node sh:path ?path .} 
        OPTIONAL {?node sh:class ?class .}
        OPTIONAL {?node sh:hasValue ?hasValue .}  

    }
    """

In [139]:
results = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
queries = [query_prop_all, query_node_all, query_node_inexplicably_working]
results = []
for i in queries: 
    for j in queries: 
        for k in queries: 
        res = g.query(k % next_node)
        res.serialize('res.csv',format = 'csv')
        print(res._bindings)
        next_node = next_node3 = res._bindings[0]['object']
        results.append(res)
    except Exception as e:
        pass
    print(next_node1,next_node2, next_node3)

[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b41'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater')}]
[]
[{rdflib.term.Variable('path'): rdflib.term.URIRef('http://data.ashrae.org/standard223#hasProperty'), rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b42')}]
https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b41 0 https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b42
[{rdflib.term.Variable('object'): rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b43'), rdflib.term.Variable('class'): rdflib.term.URIRef('http://data.ashrae.org/standard223#QuantifiableActuatableProperty')}]
[]
https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f

In [134]:
results

In [129]:
next_node = 'https://rdflib.github.io/.well-known/genid/rdflib/n3f9e2f5e271246e39102ebc895224851b44'
res = g.query(query_node_inexplicably_working % next_node)
res.serialize('res.csv',format = 'csv')

# Experimenting

In [340]:
results = []
res = g.query(query_prop_all % 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil')
results.append(res)
next_node = res._bindings[0].get('object')
while next_node:
    res = g.query(query_node_all % next_node)
    results.append(res)
    try:
        next_node = res._bindings[0].get('object')
    except Exception as e:
        print(e)
        break

IndexError: list index out of range

In [326]:
results

In [319]:
# g.query(query_prop_all % 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil').serialize('res.ttl', format = 'ttl')

In [347]:
res = g.query(query_prop_all % 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil')
res.serialize('res.csv', format = 'csv')

In [348]:
res = g.query(query_prop_all % 'https://rdflib.github.io/.well-known/genid/rdflib/nb8ec4787aa3545268c6359dff05377a6b42')
res.serialize('res.csv', format = 'csv')

In [321]:
next_node = res._bindings[0].get('next_node')
print(query % next_node)
res = g.query(query % next_node)
res.serialize('res.csv', format = 'csv')


    PREFIX s223: <http://data.ashrae.org/standard223#>
    PREFIX unit: <http://qudt.org/vocab/unit/>
    PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX g36: <http://data.ashrae.org/standard223/1.0/extensions/g36#>

    SELECT * WHERE {
        ?node rdfs:subClassOf ?what .
        OPTIONAL {?node sh:hasValue ?hasValue .}
        OPTIONAL {?node sh:class ?class .}
        OPTIONAL {?node sh:path ?path .}
        ?node sh:node*/sh:property ?next_node .
        FILTER(?node = <None>) .
    }



In [322]:
next_node = res._bindings[0].get('next_node')
print(query % next_node)
res = g.query(query % next_node)
res.serialize('res.csv', format = 'csv')

IndexError: list index out of range

In [323]:
results = []
next_node = 'http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'
while next_node:
    res = g.query(query % next_node)
    results.append(res)
    try:
        next_node = res._bindings[0].get('next_node')
    except Exception as e:
        print(e)
        break

list index out of range


In [324]:
for res in results:
    for r in res:
        print(r)

(None, rdflib.term.URIRef('https://rdflib.github.io/.well-known/genid/rdflib/n6c513dd0cc6d4c3cb42007d2ef57a027b41'), None, rdflib.term.URIRef('http://data.ashrae.org/standard223/1.0/extensions/g36#ElectricHeatingCoil'), rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater'), rdflib.term.URIRef('http://data.ashrae.org/standard223#ResistanceHeater'))
